In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# Test DFA Minimization

In this notebook, we verify our minimization algorithm using a concrete example.
We manually construct a DFA, visualize it, run the minimization, and visualize the result.

In [ ]:
import { instance } from "@viz-js/viz";
import { RecursiveSet, RecursiveMap, Tuple } from "recursive-set";

import {
    DFA,
    DFAState,
    State,
    TransRelDet
} from "./01-NFA-2-DFA";
import { minimize } from "./07-Minimize";
import { dfa2dot, dfa2string, renderLegend } from "./FSM-2-Dot";

const viz = await instance();

### Defining the DFA

We define the DFA $A = (Q, \Sigma, \delta, q_0, F)$ manually.

**Implementation Details:**
To ensure correct behavior with `RecursiveSet`, we pre-create the singleton sets representing our states ($\{0\}, \{1\}, \dots$). This allows us to cache them and ensures referential identity where useful, though v7 relies on structural equality.

**Transitions:**
* $(0, a) \to 1, \quad (0, b) \to 6$
* ... (see code for full definition)

In [ ]:
// 1. Pre-create ALL state instances (Singletons {0}..{8})
// We use a helper array to reference them easily by index
const states: DFAState[] = Array.from({ length: 9 }, (_, i) =>
    new RecursiveSet(i)
);

// Helper: Access the cached instance by index
const S = (i: number) => states[i];

const Sigma = new RecursiveSet<string>("a", "b");

// Delta uses RecursiveMap with Tuple keys
const delta = new RecursiveMap<Tuple<[DFAState, string]>, DFAState>();

const addTrans = (from: number, char: string, to: number) => {
    delta.set(new Tuple(S(from), char), S(to));
};

// Define transitions
addTrans(0, "a", 1);
addTrans(0, "b", 6);
addTrans(1, "a", 2);
addTrans(1, "b", 7);
addTrans(2, "a", 3);
addTrans(2, "b", 6);
addTrans(3, "a", 0);
addTrans(3, "b", 7);
addTrans(4, "a", 5);
addTrans(4, "b", 2);
addTrans(5, "a", 6);
addTrans(5, "b", 8);
addTrans(6, "a", 7);
addTrans(6, "b", 2);
addTrans(7, "a", 0);
addTrans(7, "b", 3);
addTrans(8, "a", 4);
addTrans(8, "b", 8);

// Construct the Sets
const Q = new RecursiveSet(...states);
const A_states = new RecursiveSet(S(2), S(3), S(6), S(7));
const q0 = S(0);

const A: DFA = {
    Q: Q,
    Σ: Sigma,
    δ: delta,
    q0: q0,
    A: A_states,
};

In [ ]:
dfa2string(A);

### Visualize Original DFA

We generate both the text representation (to see the state encoding) and the Graphviz diagram.

**Note:** The state names (S0, S1...) in the graph are generated automatically. Check the "state encoding" in the text output to map them back to our sets (e.g., see which S-name corresponds to `{0}`).

In [ ]:
const dot = dfa2dot(A);
display.html(viz.renderString(dot, { format: "svg" }));

### Minimize and Visualize

We apply the `minimize` function to compute the **Quotient Automaton**.

**Mathematical Structure:**
The result is a `MinDFA`, where each state is an **Equivalence Class** (a set of original states).
* Example: If states $\{2\}$ and $\{6\}$ are equivalent, the new state is the set `{{2}, {6}}`.

In [ ]:
const F = minimize(A);

In [ ]:
dfa2string(F);

### Visualize Minimized DFA

Now we can use our standard tools without extra casting.

The output shows the merged states (e.g., `{{3}, {7}}`).

In [ ]:
const dot2 = dfa2dot(F);
display.html(viz.renderString(dot2, { format: "svg" }));

In [ ]:
display.html(renderLegend(F))